In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

lookback = 1200
horizon = 240

cpu


In [ ]:
base_dir = "../AI-Model-Artifacts/"
print("Base directory:", base_dir)

if os.path.exists(base_dir):
    print("✅ Directory exists and is accessible.")
    items = os.listdir(base_dir)
    print("Items in directory:")
    for item in items:
        print(f"  - {item}")
else:
    print("❌ Directory does not exist or is not accessible.")


Base directory: ../AI-Model-Artifacts/
✅ Directory exists and is accessible.
Items in directory:
  - MinMax_Scaler.pkl
  - model_best
  - multi_vanilla_patchtst.pt
  - Scaled_Test.npy
  - Test.csv


In [9]:
model_path  = os.path.join(base_dir, "multi_vanilla_patchtst.pt")
scaler_path = os.path.join(base_dir, "MinMax_Scaler.pkl")
X_test_path = os.path.join(base_dir, "X_test.npy")                  # Data From the DB
print("🔄 Files Loaded")


🔄 Files Loaded


In [10]:
scaled_X_test = np.load(X_test_path)
print("✅ Data loaded:", scaled_X_test.shape)

X_last = scaled_X_test
X_tensor = torch.tensor(X_last, dtype=torch.float32).unsqueeze(0).to(device)
print("✅ Input tensor shape:", X_tensor.shape)  # shape (1, 1200, 4)


✅ Data loaded: (5560, 1200, 4)
✅ Input tensor shape: torch.Size([1, 5560, 1200, 4])


In [13]:
# Load the scaler
with open(scaler_path, "rb") as f:
    scaler = pickle.load(f)

# Load the model
print(f"✅ Using device: {device}")
model = torch.load(model_path, map_location=device, weights_only=False)
model.to(device)
model.eval()
print("✅ PatchTST model loaded successfully!")


✅ Using device: cpu


f:\### STUDIES ###\# Computer Engineering\Sem_08\# FYP\PROJECT\Workflow\Workflow_Test_1\FYP-Machine-Condition-Prediction\_fyp_wf_1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ PatchTST model loaded successfully!


In [ ]:
# Scale Or Inverse Scale
# Choose what's suitable
# Runs up to here without errors

In [ ]:
# Forecast
with torch.no_grad():
    outputs = model(past_values=X_tensor)
    forecast = outputs.prediction_outputs.squeeze().cpu().numpy()

print("✅ Raw forecast stats:")
print(f"Shape: {forecast.shape}")
print(f"Min: {forecast.min()}, Max: {forecast.max()}")
print(f"NaN count: {np.isnan(forecast).sum()}")
print(f"NaN per feature: {np.isnan(forecast).sum(axis=0)}")


In [ ]:
num_features = X_last.shape[1] if X_last.ndim == 2 else 1

for feature_idx in range(num_features):
    # Select single feature data
    X_feature = X_last[:, feature_idx] if X_last.ndim == 2 else X_last
    forecast_feature = forecast[:, feature_idx] if forecast.ndim == 2 else forecast

    # Inverse transform to original scale
    forecast_feature_scaled = forecast_feature.reshape(-1, 1)
    feature_min = scaler.min_[feature_idx]
    feature_scale = scaler.scale_[feature_idx]
    forecast_feature_orig = forecast_feature_scaled * (1/feature_scale) - (feature_min/feature_scale)
    forecast_feature_orig = forecast_feature_orig.flatten()

    X_feature_scaled = X_feature.reshape(-1, 1)
    X_feature_orig = X_feature_scaled * (1/feature_scale) - (feature_min/feature_scale)
    X_feature_orig = X_feature_orig.flatten()

    # Compute Tolerance Band in original scale
    lowest_actual = np.min(X_feature_orig)
    highest_actual = np.max(X_feature_orig)
    band_half = (highest_actual - lowest_actual) / 10
    band_lower = forecast_feature_orig - band_half
    band_upper = forecast_feature_orig + band_half
    print(f"✅ Feature_{feature_idx}: Computed tolerance band ±{band_half:.4f} (Original Scale)")

    # Plot in original scale
    plt.figure(figsize=(14, 6))

    plt.plot(
        np.arange(lookback, lookback + horizon),
        X_feature_orig,
        color="green",
        linestyle="--",
        marker="o",
        label="Actual Lookback",
    )
    plt.plot(
        np.arange(lookback, lookback + len(forecast_feature_orig)),
        forecast_feature_orig,
        color="red",
        linestyle=":",
        marker="x",
        label="Model Forecast",
    )
    plt.fill_between(
        np.arange(lookback, lookback + len(forecast_feature_orig)),
        band_lower,
        band_upper,
        color='red',
        alpha=0.2,
        label="Forecast Tolerance Band"
    )
    plt.xlabel("Time Steps")
    plt.ylabel("Feature Value (Original Scale)")
    plt.title(f"Feature_{feature_idx} Forecast — Actual vs Predicted Horizon with Tolerance Band")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()